# Clasificador de textos políticos - GEMINI

In [1]:
%pip install google-generativeai
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
import os
import json
import google.generativeai as genai
from dotenv import load_dotenv

# === CONFIGURACIÓN ===
CARPETA_ENTRADA = "textos"
CARPETA_SALIDA = "resultados"
ARCHIVO_LOG = "log_errores.txt"

# Inicializar modelo
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

# Inicializar modelo con parámetros ajustados para mejorar precisión y robustez
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.8,
        "top_p": 0.8,
        "top_k": 40,
        "candidate_count": 1
    }
)

In [3]:
# Asegurar que exista la carpeta de salida
os.makedirs(CARPETA_SALIDA, exist_ok=True)

# Inicializar archivo de log
with open(ARCHIVO_LOG, "w", encoding="utf-8") as log:
    log.write("=== LOG DE ERRORES ===\n")

# Obtener archivos ya procesados
procesados = {
    os.path.splitext(f)[0]
    for f in os.listdir(CARPETA_SALIDA)
    if f.endswith(".json")
}    

# Archivos de entrada (excluyendo los ya procesados)
archivos_txt = []
for raiz, _, archivos in os.walk(CARPETA_ENTRADA):
    for archivo in archivos:
        if archivo.endswith(".txt"):
            nombre_base = os.path.splitext(archivo)[0]
            if nombre_base not in procesados:
                archivos_txt.append(os.path.join(raiz, archivo))


# Token acumulado
total_tokens_global = 0

In [4]:
def interpretar_score(score):
    if score < 0.2:
        return "ultraizquierda"
    elif score < 0.4:
        return "izquierda moderada"
    elif score < 0.6:
        return "neutral"
    elif score < 0.8:
        return "derecha moderada"
    else:
        return "ultraderecha"

In [5]:
# Procesamiento
for ruta_entrada in archivos_txt:
    archivo = os.path.basename(ruta_entrada)
    subcarpeta = os.path.basename(os.path.dirname(ruta_entrada))  # Esta será la etiqueta_original
    ruta_salida = os.path.join(CARPETA_SALIDA, f"{os.path.splitext(archivo)[0]}.json")

    with open(ruta_entrada, "r", encoding="utf-8") as f:
        texto = f.read()

    prompt = f"""
    Vas a analizar un texto con orientación política en Argentina. Devolvé una respuesta en formato JSON con la siguiente estructura exacta:

    {{
      "texto_id": "nombre_del_texto",
      "etiqueta_original": "{subcarpeta}",
      "perfil_global_del_texto": {{
        "score_promedio": "",
        "interpretacion": ""
      }},
      "frases": [
        {{
          "frase": "frase completa",
          "score": float entre 0 y 1 indicando cercanía a derecha (0 = izquierda, 1 = derecha),
          "etiqueta": "izquierda|derecha|neutral"
        }},
        ...
      ]
    }}

    Reglas:
    - Extraé frases significativas (mínimo 3).
    - Clasificá cada frase individualmente.
    - Usá el nombre del archivo sin extensión como "texto_id".

    Texto a analizar:
    \"\"\"{texto}\"\"\"
    """

    try:
        response = model.generate_content(prompt)

        # Si el prompt fue bloqueado por Gemini (por contenido sensible u otra política)
        if not response.candidates:
            block_reason = getattr(response.prompt_feedback, "block_reason", "UNKNOWN")
            raise ValueError(f"Prompt bloqueado por política: {block_reason}")

        raw = response.candidates[0].content.parts[0].text.strip()
        if raw.startswith("```json"):
            raw = raw.removeprefix("```json").removesuffix("```").strip()

        total_tokens = getattr(response.usage_metadata, "total_token_count", None)
        if total_tokens:
            total_tokens_global += total_tokens

        resultado = json.loads(raw)

        # Calcular promedio de score y asignar interpretación
        frases = resultado.get("frases", [])
        if frases:
            promedio = sum(f.get("score", 0) for f in frases) / len(frases)
            resultado["perfil_global_del_texto"] = {
                "score_promedio": round(promedio, 3),
                "interpretacion": interpretar_score(promedio)
            }

        if "texto_id" not in resultado:
            resultado["texto_id"] = os.path.splitext(archivo)[0]

        resultado["tokens_procesados"] = total_tokens

        with open(ruta_salida, "w", encoding="utf-8") as f:
            json.dump(resultado, f, ensure_ascii=False, indent=2)

        print(f"✅ Procesado: {archivo} | Tokens: {resultado.get('tokens_procesados', 'N/A')}")

    except Exception as e:
        total_tokens = getattr(response.usage_metadata, "total_token_count", None)
        resultado = {
            "texto_id": os.path.splitext(archivo)[0],
            "error": str(e),
            "raw_response": "",  # ⚠️ No accedemos a .text si fue bloqueado
            "tokens_procesados": total_tokens
        }
        if total_tokens:
            total_tokens_global += total_tokens
        with open(ARCHIVO_LOG, "a", encoding="utf-8") as log:
            log.write(f"❌ Error en {archivo}: {str(e)}\n")

print(f"\n🧠 Total global de tokens procesados: {total_tokens_global}")




✅ Procesado: Es un manicomio - Editorial #VivianaConVos 27072022.txt | Tokens: 8615
✅ Procesado: Falta de coraje - Editorial #VivianaConVos 25072022.txt | Tokens: 8717
✅ Procesado: Larreta decidió separar boletas y M. Macri está desilusionado  Apertura MF.txt | Tokens: 4025
✅ Procesado: Libertad de expresión #VivianaConVos 08082022.txt | Tokens: 3068
✅ Procesado: Nada es gratis - Editorial #VivianaConVos 02082022.txt | Tokens: 8657
✅ Procesado: Paren de robar - Editorial #VivianaConVos 01082022.txt | Tokens: 7432
✅ Procesado: Parole, parole, parole - Editorial #VivianaConVos 26072022.txt | Tokens: 8678


KeyboardInterrupt: 

In [5]:
import os
import json

def evaluar_predicciones(directorio):
    total = 0
    aciertos = 0
    errores = []

    for raiz, _, archivos in os.walk(directorio):
        for archivo in archivos:
            if archivo.endswith(".json"):
                path = os.path.join(raiz, archivo)
                try:
                    with open(path, encoding="utf-8") as f:
                        data = json.load(f)

                    etiqueta = data.get("etiqueta_original", "").lower()
                    interpretacion = data.get("perfil_global_del_texto", {}).get("interpretacion", "").lower()

                    total += 1
                    if etiqueta in interpretacion:
                        aciertos += 1
                    else:
                        errores.append({
                            "archivo": archivo,
                            "etiqueta_original": etiqueta,
                            "interpretacion": interpretacion
                        })
                except Exception as e:
                    print(f"⚠️ Error procesando {archivo}: {e}")

    if total == 0:
        print("❌ No se encontraron archivos JSON.")
        return

    precision = round(aciertos / total * 100, 2)
    print(f"✅ Total archivos analizados: {total}")
    print(f"🎯 Predicciones correctas: {aciertos}")
    print(f"❌ Predicciones incorrectas: {total - aciertos}")
    print(f"📊 Accuracy: {precision}%")

    return {
        "total": total,
        "aciertos": aciertos,
        "errores": errores,
        "Accuracy": precision
    }

In [6]:
resultados = evaluar_predicciones("resultados")

✅ Total archivos analizados: 3100
🎯 Predicciones correctas: 2587
❌ Predicciones incorrectas: 513
📊 Accuracy: 83.45%


# Utils - no necesarias para el clasificador

In [ ]:
import os
import json

def eliminar_jsons_con_error(carpeta_salida):
    """
    Elimina todos los archivos JSON dentro del directorio dado que contengan la clave "error".

    Parámetros:
    - carpeta_salida (str): Ruta al directorio donde están los JSON procesados.

    Devuelve:
    - int: Cantidad de archivos eliminados.
    """
    eliminados = 0

    for archivo in os.listdir(carpeta_salida):
        if archivo.endswith(".json"):
            ruta = os.path.join(carpeta_salida, archivo)
            try:
                with open(ruta, encoding="utf-8") as f:
                    data = json.load(f)
                if "error" in data:
                    os.remove(ruta)
                    eliminados += 1
                    print(f"🗑️ Eliminado: {archivo}")
            except Exception as e:
                print(f"⚠️ No se pudo analizar {archivo}: {e}")

    print(f"\n✅ Total de archivos eliminados: {eliminados}")
    return eliminados


In [ ]:
eliminar_jsons_con_error(CARPETA_SALIDA)
